# notifying changes in computational notebooks

blind and low vision jupyter users frequently ask for audible (non-visual) announcements about changes to the state of the document. this demand introduces a new visual and nonvisual feature to computational notebooks that notify any user of an update.

this feature request starts as a nonvisual componment, but quickly we realize that a visual component would be assistive to all users.

options for notifications: toast (visual), aria-live

[demo!](#log)

In [1]:
from nbconvert_a11y.tables import get_table, Config, new
shell.tangle.parser = midgy.language.python.Python()

https://stackoverflow.com/questions/2031163/when-to-use-the-different-log-levels

screen reader users should be able to configure aria live settings.
a BVI developer may want to hear debug messages.s

In [2]:
%%
## creating a scenario that would be announced in a log

    from logging import FATAL, CRITICAL, ERROR, WARN, INFO, DEBUG
    FATAL / CRITICAL, ERROR, WARN, INFO, DEBUG;

    activities =\
1. info
    : starting python kernel
    : - show kernel info
2. info
    : python kernel started
    : - execute cells
3. info
    : cell 2 executed
    : - cancel cell 2 execution
4. info
    : cell 2 finished successfully
    : - jump to cell output
5. info
    : cell 7 executed
    : - cancel cell 7 execution
6. error
    : cell 7 failed with TypeError
    : - jump to cell 7 traceback
7. critical
    : python kernel restarting
    : - queue cell execution
8. critical
    : python kernel restarted
    : - execute cells

In [3]:
%%
we need to map `logging` level messages to `aria-live` .

    logging_roles =\
```yaml
ERROR: assertive
CRITICAL: assertive
INFO: polite
WARN: none
DEBUG: none
```

## creating a dataframe about our synthetic scenario

In [4]:
df = pipe(
    activities, partial(re.sub, "\s+:", ":"), 
    partial(re.sub, "[0-9].\s+|\s-\s", ""), str.splitlines, Series
).str.partition(": ")[[0, 2]].rename(columns=pipe(
    "level message".split(), partial(zip, range(0, 3, 2)), dict, 
))

df["timestamp"] =df.level.apply(
    lambda x: random.randint(1, 10)
).cumsum().pipe(
    pandas.to_datetime, unit="s"
)
df = df.set_index("timestamp")
df["level"] = df["level"].str.upper()

df["live"] = df["level"].apply(logging_roles.get)
df = df.drop(columns="message").assign(
    **df.message.str.partition(":")[[0, 2]].rename(columns=pipe(
        "message action".split(), partial(zip, range(0, 3, 2)), dict, 
    ))
)
df["id"] = [uuid.uuid4() for _ in range(len(df))]

pipe(df, partial(get_table, config=Config(), caption="information needed to construct an accessible table representation"))

timestamp,level,live,message,action,id
1970-01-01 00:00:01,INFO,polite,starting python kernel,show kernel info,95752cd2-b8d9-46ec-911a-9ea19aaa63be
1970-01-01 00:00:04,INFO,polite,python kernel started,execute cells,f02cd379-a4e1-4121-8d92-e1573dbcb702
1970-01-01 00:00:07,INFO,polite,cell 2 executed,cancel cell 2 execution,1be87cc3-2007-401d-a235-68bb9eb0ce9a
1970-01-01 00:00:12,INFO,polite,cell 2 finished successfully,jump to cell output,1220998b-f07d-4061-983d-69417a1c7df1
1970-01-01 00:00:17,INFO,polite,cell 7 executed,cancel cell 7 execution,978e832a-5ecf-419f-bbe4-2920babb9a2f
1970-01-01 00:00:26,ERROR,assertive,cell 7 failed with TypeError,jump to cell 7 traceback,556b95ea-34e6-46cc-b7b2-bbb7efd85e0a
1970-01-01 00:00:35,CRITICAL,assertive,python kernel restarting,queue cell execution,7e8baa0d-a555-4bdd-9ea8-63052a9d51fe
1970-01-01 00:00:44,CRITICAL,assertive,python kernel restarted,execute cells,13c17ac0-881b-42dd-8c7f-d0f3f8c13300
timestamp,level,live,message,action,id
min,CRITICAL,assertive,cell 2 executed,cancel cell 2 execution,1220998b-f07d-4061-983d-69417a1c7df1


## accessible table components

In [5]:
# https://developer.mozilla.org/en-US/docs/Web/HTML/Element/time

def get_timestamp(s):
    t = new("time", attrs=dict(datetime=s.name.isoformat()))
    # t.append(s.name.strftime('%Y-%m-%d %X'))
    t.append(s.name.strftime('%X'))
    return t

there are a lot of [shenangins to consider when implementing aria live across browsers](https://a11ysupport.io/tech/aria/aria-live_attribute).
we take an html5 forward design approach and restrict ourselves to the nuance around the native output element;
we take heavy advice from [scott o'hara's html `output` analysis](https://scottaohara.github.io/tests/html-output/).

In [6]:
def get_message(s):
    object = new("output", attrs=dict(
        role="status", id=s.loc["id"], 
        onload="alert(11);",  **{"aria-live": s.live, "data-message": s.message}
    ))
    # object.append(s.message)
    return object

aria live seems most reliable when the text is injected directly into the element

In [7]:
def get_script(s):
    script = new("script")
    script.append(F""" setTimeout(() => {{
            var out = document.getElementById("{s.id}");
            out.parentElement.parentElement.classList.remove("nv");
            out.textContent = out.dataset.message}}, {s.name.second * 1000})""")
    return script

In [8]:
def get_priority(s):
    label = new("label", attrs={"for": s.loc["id"]})
    label.append(s.loc["level"])
    return label

a visual convention with toast notifications is to provide a call to action with a notification.
we couple these visual concepts with non visual concepts.
jupyter has toast notifications, but they would be compliments the activity log.
the activity log is the domain of visual and nonvisual actions.

In [ ]:
def get_action(s):
    # might return a link or a button
    butt = new("a" if s.action.startswith(("jump",)) else "button")
    butt.append(s.action)
    return butt

an example activity log `table` that reveals it self and has aria live capabilities.
this is a first demonstration of a more robust activity log for visual and nonvisual users.
the table operates as a keyboard navigable component on assistive technology.

In [ ]:
table = (t := df.apply(
    lambda s: Series(dict(
        priority=get_priority(s),
        message=get_message(s),
        action=get_action(s), 
        timestamp=get_timestamp(s), script=get_script(s)
    )), axis=1
)).reset_index(drop=True).set_index("timestamp").pipe(get_table, id="log", **{"aria-labelledby": "log-nm"})
[x.attrs.update(**{"class": "nv"}) for x in table.select("tr")];

the table doesn't need to be seen, but it can not be removed from the visual domain otherwise announcements will be suppressed.
we can use the `details` tag and the non visual css styling to address this.
by wrapping all of the following in a region we expose a landmark for the activity log enhancing discovery.

In [ ]:
section = new("section", attrs={"aria-labelledby": "log-nm"})
section.append(details := new("details", attrs=dict(open="")))
details.append(summary := new("summary", attrs=dict(id="log-nm")))
summary.append("activity log")
section.append(table)
section

## styling

In [ ]:
%%
<figure>
<figcaption>
<a href="https://www.tpgi.com/the-anatomy-of-visually-hidden/">visually hidden css</a></figcaption>
<blockquote cite="https://www.tpgi.com/the-anatomy-of-visually-hidden/">
    
    display\
```css
details:not([open]) + table, .nv {
    clip: rect(0 0 0 0);
    clip-path: inset(50%);
    height: 1px;
    overflow: hidden;
    position: absolute;
    white-space: nowrap;
    width: 1px;
}
a[href="#log"] {
    display: block;
    height: 300px;
    width: 400px;
    background: -moz-element(#log);
    background-size: content;
    background-repeat: no-repeat;
    font-size: 4rem;
}
```
</blockquote>
</figure>